In [ ]:
# JWST Data Quality Flags: Bit Position to Mnemonic Name
dq_flags = {
    0: "DO\_NOT\_USE",           # 2^0 = 1
    1: "SATURATED",            # 2^1 = 2
    2: "JUMP\_DET",             # 2^2 = 4
    3: "DROPOUT",              # 2^3 = 8
    4: "OUTLIER",              # 2^4 = 16
    5: "PERSISTENCE",          # 2^5 = 32
    6: "AD\_FLOOR",             # 2^6 = 64
    7: "CHARGELOSS",           # 2^7 = 128
    8: "UNRELIABLE\_ERROR",     # 2^8 = 256
    9: "NON\_SCIENCE",          # 2^9 = 512
    10: "DEAD",                # 2^10 = 1024
    11: "HOT",                 # 2^11 = 2048
    12: "WARM",                # 2^12 = 4096
    13: "LOW\_QE",              # 2^13 = 8192
    14: "RC",                  # 2^14 = 16384
    15: "TELEGRAPH",           # 2^15 = 32768
    16: "NONLINEAR",           # 2^16 = 65536
    17: "BAD\_REF_PIXEL",       # 2^17 = 131072
    18: "NO\_FLAT\_FIELD",       # 2^18 = 262144
    19: "NO\_GAIN\_VALUE",       # 2^19 = 524288
    20: "NO\_LIN\_CORR",         # 2^20 = 1048576
    21: "NO\_SAT\_CHECK",        # 2^21 = 2097152
    22: "UNRELIABLE\_BIAS",     # 2^22 = 4194304
    23: "UNRELIABLE\_DARK",     # 2^23 = 8388608
    24: "UNRELIABLE\_SLOPE",    # 2^24 = 16777216
    25: "UNRELIABLE\_FLAT",     # 2^25 = 33554432
    26: "OPEN",                # 2^26 = 67108864
    27: "ADJ\_OPEN",            # 2^27 = 134217728
    28: "FLUX\_ESTIMATED",      # 2^28 = 268435456
    29: "MSA\_FAILED\_OPEN",     # 2^29 = 536870912
    30: "OTHER\_BAD\_PIXEL",     # 2^30 = 1073741824
    31: "REFERENCE\_PIXEL"      # 2^31 = 2147483648
}

In [ ]:
import numpy as np
import pickle
from astropy.io import fits
import concurrent.futures
import shutil

from joblib import Parallel, delayed
import multiprocessing
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit
from scipy.ndimage import uniform_filter, label
import warnings
from sklearn.metrics import r2_score

import scienceplots
plt.style.use(['science'])

In [ ]:
import glob
project_no = 1224
observation = 3
nrs_no = 2

#f'../Data/data_0{project_no}/Obs00{observation}/uncal/' + 

ramp_files_modified = sorted(glob.glob(f'../Data/data_0{project_no}/Obs00{observation}/uncal/' + f'*jw0{project_no}*' + f'*nrs{nrs_no}*' + '*ramp_m*'))
print(ramp_files_modified)

In [ ]:
with fits.open(ramp_files_modified[0]) as hdul:
    raw_data = hdul[3].data  # Assuming shape is already (150, 60, height, width)

# Sanity check
print("Raw data shape:", raw_data.shape)

data_cube = raw_data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os
from numba import njit, prange

@njit(parallel=True)
def compute_counts(data_cube):
    """
    Use Numba to compute bincounts per value over time steps.
    data_cube shape: (N, T, H, W)
    Returns: (32, T) array of counts
    """
    N, T, H, W = data_cube.shape
    counts_per_value = np.zeros((32, T), dtype=np.int64)

    for t in prange(T):
        temp = np.zeros(32, dtype=np.int64)
        for n in range(N):
            for h in range(H):
                for w in range(W):
                    val = data_cube[n, t, h, w]
                    temp[val] += 1
        for v in range(32):
            counts_per_value[v, t] = temp[v]

    return counts_per_value

def plot_comparison_across_files(file_paths, labels=None, colors=None):
    """
    Compare histogram value trends over time steps from multiple FITS files.
    """
    assert len(file_paths) > 0, "Provide at least one FITS file path."
    if labels is None:
        labels = [os.path.basename(fp).replace('.fits', '') for fp in file_paths]
    if colors is None:
        colors = ['blue', 'red', 'green', 'olive', 'orange', 'cyan', 'purple', 'brown']

    counts_all_files = []

    for file_path in file_paths:
        with fits.open(file_path) as hdul:
            data_cube = hdul[3].data.astype(np.int64)  # Ensure integer type for Numba
            counts_per_value = compute_counts(data_cube)
            counts_all_files.append(counts_per_value)

    total_counts = sum(counts for counts in counts_all_files)
    nonzero_values = np.where(total_counts.sum(axis=1) > 0)[0]
    nonzero_values = nonzero_values[nonzero_values != 0]  # skip zero if desired

    num_timesteps = counts_all_files[0].shape[1]
    num_plots = len(nonzero_values)
    cols = 4
    rows = int(np.ceil(num_plots / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 3))
    axes = axes.flatten()

    for idx, value in enumerate(nonzero_values):
        ax = axes[idx]
        for file_idx, counts in enumerate(counts_all_files):
            ax.plot(
                counts[value],
                label=labels[file_idx].split('01-')[1][:11],
                color=colors[file_idx % len(colors)]
            )
        ax.set_title(f"Pixel type: $\\texttt{{{dq_flags.get(value, str(value))}}}$")
        ax.set_xlim(0, num_timesteps - 1)
        ax.set_xticks([0, num_timesteps//2, num_timesteps - 1])
        ax.set_xlabel('Group')
        ax.set_ylabel('Count')

    for j in range(num_plots, len(axes)):
        axes[j].axis('off')

    # Shared legend outside plot area
    fig.legend(
        labels=[label.split('01-')[1][:11].replace('_', r'\_') for label in labels],
        loc='center right',
        fontsize='small',
        frameon=False
    )

    plt.tight_layout(rect=[0, 0, 0.85, 1])  # Make space on right for legend
    plt.savefig(f'Data0{project_no}Obs00{observation}nrs{nrs_no}_count.pdf')
    plt.show()

# Call your function as before
plot_comparison_across_files(ramp_files_modified)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from astropy.io import fits
from numba import njit, prange

# --- Numba-accelerated histogram function ---
@njit(parallel=True)
def compute_counts_per_value(data_cube):
    """
    Compute histogram counts for values 0–31 across all time steps.
    data_cube shape: (slices, time, height, width)
    Returns: array of shape (32, num_timesteps)
    """
    slices, num_timesteps, height, width = data_cube.shape
    counts_per_value = np.zeros((32, num_timesteps), dtype=np.int64)

    for t in prange(num_timesteps):
        temp_counts = np.zeros(32, dtype=np.int64)
        for s in range(slices):
            for h in range(height):
                for w in range(width):
                    val = data_cube[s, t, h, w]
                    temp_counts[val] += 1
        for v in range(32):
            counts_per_value[v, t] = temp_counts[v]

    return counts_per_value


# --- Main code execution ---
with fits.open(ramp_files_modified[0]) as hdul:
    data_cube = hdul[3].data.astype(np.int64)  # Ensure int64 for Numba compatibility

num_slices, num_timesteps, height, width = data_cube.shape
print(f"Data cube shape: {data_cube.shape}")

# Compute histogram counts using optimized function
counts_per_value = compute_counts_per_value(data_cube)

# Identify nonzero pixel values (excluding 0 if desired)
nonzero_values = np.where(counts_per_value.sum(axis=1) > 0)[0]
nonzero_values = nonzero_values[nonzero_values != 0]
num_plots = len(nonzero_values)

# Set up plot grid
cols = 4
rows = int(np.ceil(num_plots / cols))
fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 2.5))
axes = axes.flatten()

# Plot bar charts
for idx, value in enumerate(nonzero_values):
    ax = axes[idx]
    ax.bar(
        range(num_timesteps),
        counts_per_value[value],
        color=(mcolors.to_rgba('blue', alpha=0.3)),
        edgecolor=mcolors.to_rgba('blue', alpha=0.7),
        linewidth=1
    )
    ax.set_title(f"Value {value}")
    ax.set_xlim(0, num_timesteps - 1)
    ax.set_xticks([0, num_timesteps // 2, num_timesteps - 1])

# Hide unused subplots
for j in range(num_plots, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from numba import njit, prange
from astropy.io import fits
import os

# Numba-accelerated function to detect streaks allowing a single zero or 30 in between
@njit
def get_streaks_allowing_single_zero_numba(arr, target_value):
    n = arr.shape[0]
    max_streaks = n  # max possible streaks
    starts = np.empty(max_streaks, dtype=np.int32)
    ends = np.empty(max_streaks, dtype=np.int32)
    lengths = np.empty(max_streaks, dtype=np.int32)

    count = 0
    i = 0

    while i < n:
        if arr[i] == target_value:
            start = i
            length = 1
            gaps = 0
            i += 1

            while i < n:
                if arr[i] == target_value:
                    length += 1
                elif (arr[i] == 0 or arr[i] == 30) and (i + 1 < n and arr[i + 1] == target_value) and gaps == 0:
                    length += 1  # count zero as part of streak
                    gaps += 1
                    i += 1  # skip the zero
                    length += 1  # count next target_value
                else:
                    break
                i += 1

            end = start + length - 1
            starts[count] = start
            ends[count] = end
            lengths[count] = length
            count += 1
        else:
            i += 1

    return starts[:count], ends[:count], lengths[:count]

# Numba-parallelized function to process a single slice and find streaks for one target value
@njit(parallel=True)
def process_slice_for_value_numba(data_cube_slice, target_value):
    # data_cube_slice shape: (time, rows, cols)
    time_len, rows, cols = data_cube_slice.shape

    max_streaks = time_len * rows * cols  # pessimistic max
    starts = np.empty(max_streaks, dtype=np.int32)
    ends = np.empty(max_streaks, dtype=np.int32)
    lengths = np.empty(max_streaks, dtype=np.int32)

    count = 0

    for col in prange(cols):
        for row in range(rows):
            streak_array = data_cube_slice[:, row, col]
            # Check if target_value present
            found = False
            for t in range(time_len):
                if streak_array[t] == target_value:
                    found = True
                    break
            if not found:
                continue

            s, e, l = get_streaks_allowing_single_zero_numba(streak_array, target_value)
            for idx in range(s.shape[0]):
                if count < max_streaks:
                    starts[count] = s[idx]
                    ends[count] = e[idx]
                    lengths[count] = l[idx]
                    count += 1

    return starts[:count], ends[:count], lengths[:count]

def analyze_streaks_across_files_numba(file_paths, nonzero_values, labels=None):
    """
    Analyze streaks of target values across columns and slices in multiple FITS files.

    Parameters:
    - file_paths: list of str, paths to FITS files.
    - nonzero_values: list or array of int values to search for streaks.
    - labels: optional list of labels corresponding to file_paths.

    Returns:
    - Dictionary with structure:
      {
          'label1': {value1: streaks_array, value2: streaks_array, ...},
          'label2': {...},
          ...
      }
    Each streaks_array is a numpy structured array with fields ('start', 'end', 'length').
    """

    if labels is None:
        labels = [os.path.basename(fp).replace('.fits', '') for fp in file_paths]

    results_by_file = {}

    for file_path, label in zip(file_paths, labels):
        print(f"\n📂 Processing file: {label}")

        with fits.open(file_path) as hdul:
            data_cube = hdul[3].data  # shape (num_slices, time, rows, cols)
            num_slices = data_cube.shape[0]

        file_results = {}
        for val in nonzero_values:
            print(f"  ➤ Target value {val}...")
            all_starts = []
            all_ends = []
            all_lengths = []

            for slice_idx in range(num_slices):
                data_cube_slice = data_cube[slice_idx]  # shape (time, rows, cols)
                s, e, l = process_slice_for_value_numba(data_cube_slice, val)
                all_starts.append(s)
                all_ends.append(e)
                all_lengths.append(l)

            # Concatenate arrays for all slices
            if all_starts:
                all_starts = np.concatenate(all_starts)
                all_ends = np.concatenate(all_ends)
                all_lengths = np.concatenate(all_lengths)
            else:
                all_starts = np.array([], dtype=np.int32)
                all_ends = np.array([], dtype=np.int32)
                all_lengths = np.array([], dtype=np.int32)

            # Create structured array for easy use
            streaks_array = np.empty(all_starts.shape[0], dtype=[('start', np.int32), ('end', np.int32), ('length', np.int32)])
            streaks_array['start'] = all_starts
            streaks_array['end'] = all_ends
            streaks_array['length'] = all_lengths

            file_results[val] = streaks_array

        results_by_file[label] = file_results

    return results_by_file

# Call the Numba-optimized function
results_by_file = analyze_streaks_across_files_numba(
    ramp_files_modified,
    nonzero_values
)

In [ ]:
# Open the FITS file and display the header and XPOSURE value
with fits.open(ramp_files_modified[0]) as hdul:
    header = hdul[0].header  # Access the primary HDU header
    tgroup_value = header.get('TGROUP', 'TGROUP key not found')
    print("TGROUP value:", tgroup_value)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Store all percentages for averaging

n_plots = len(nonzero_values)
n_rows = 4
n_cols = int(np.ceil(n_plots / n_rows))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
axes = axes.flatten()

for i, val in enumerate(nonzero_values):
    ax = axes[i]
    all_percentages = {}

    for files in ramp_files_modified:
        data = results_by_file[files.split('/')[-1].replace('.fits', '')][val]
    
        if len(data) == 0:
            continue
        else: 
            data = data['length']
            data = data[data > 0]  # Filter out zero lengths
            data = data[data < 70]  # Filter out very long lengths
        unique, counts = np.unique(data, return_counts=True)
        percentages = counts / counts.sum()
        
        for u, p in zip(unique, percentages):
            if u not in all_percentages:
                all_percentages[u] = []
            all_percentages[u].append(p)

    # Sort keys and compute average and std dev
    sorted_keys = sorted(all_percentages.keys())
    averages = np.array([np.mean(all_percentages[k]) for k in sorted_keys])
    stds = np.array([np.std(all_percentages[k]) for k in sorted_keys])

    # Plotting
    ax.bar(sorted_keys, averages, yerr=stds, capsize=2, color='skyblue', edgecolor='blue', alpha=0.7, label='Average with Std Dev')
    ax.set_xlabel(r'$N_\text{group}$ unreliability length')
    ax.set_ylabel('Percentage occurrence')
    ax.set_title(f"Pixel type: $\\texttt{{{dq_flags[val]}}}$")

# Remove empty plots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')
plt.savefig(f'Data0{project_no}Obs00{observation}nrs{nrs_no}_bar_plot.pdf')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit

def exponential(x, a, b):
    return a * np.exp(-b * (x - 1))

fit_results = []  # To store (val, file, popt, r2score)

nonzero_values_fix = [11, 13, 26, 27, 30]

n_plots = len(nonzero_values_fix)
n_rows = 3
n_cols = int(np.ceil(n_plots / n_rows))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
axes = axes.flatten()

for i, val in enumerate(nonzero_values_fix):
    ax = axes[i]
    for files in ramp_files_modified:
        data = results_by_file[files.split('/')[-1].replace('.fits', '')][val]
        if len(data) == 0:
            continue
        data = data['length']
        data = data[data > 0]
        data = data[data < 70]
        unique, counts = np.unique(data, return_counts=True)
        percentages = counts / counts.sum()
        
        # Exclude the last value for fitting
        unique_fit = unique[:-1] * tgroup_value
        percentages_fit = percentages[:-1]
        
        try:
            popt, pcov = curve_fit(exponential, unique_fit, percentages_fit)
            perr = np.sqrt(np.diag(pcov))  # Standard deviations of the parameters
            r2 = r2_score(percentages_fit, exponential(unique_fit, *popt))
            fit_results.append([popt[0], popt[1], r2])
            ax.bar(unique * tgroup_value, percentages)
            ax.plot(
                unique_fit, 
                exponential(unique_fit, *popt), 
                label=(
                    f'$a={popt[0]:.2f} \pm {perr[0]:.2f}, '
                    f'b={popt[1]:.2f} \pm {perr[1]:.2f}, '
                    f'R^2={r2:.2f}$'
                )
            )
        except Exception as e:
            continue
    ax.set_xlabel(r'Length of time pixel stayed unreliable [s]')
    ax.set_ylabel('Percentage occurrence')
    ax.set_title(f"Pixel type: {dq_flags[val]}")
    ax.legend()

for j in range(i + 1, len(axes)):
    axes[j].axis('off')
plt.show()


In [ ]:
from tabulate import tabulate

# Create the table as before
grouped = [fit_results[i:i+2] for i in range(0, len(fit_results), 2)]
table_data = []
for label, group in zip(nonzero_values_fix, grouped):
    a_vals = [arr[0] * 100 for arr in group] 
    b_vals = [arr[1] for arr in group]
    r2_vals = [arr[2] for arr in group]
    table_data.append([
        f"$\\texttt{{{dq_flags[label]}}}$",
        f"{np.mean(a_vals):.3f} $\pm$ {np.std(a_vals):.3f}",
        f"{np.mean(b_vals):.3f} $\pm$ {np.std(b_vals):.3f}",
        f"{np.mean(r2_vals):.3f} $\pm$ {np.std(r2_vals):.3f}"
    ])

# Add overall mean row
all_a = [arr[0] * 100 for arr in fit_results]
all_b = [arr[1] for arr in fit_results]
all_r2 = [arr[2] for arr in fit_results]
table_data.append([
    "Overall mean",
    f"{np.mean(all_a):.3f} $\pm$ {np.std(all_a):.3f}",
    f"{np.mean(all_b):.3f} $\pm$ {np.std(all_b):.3f}",
    f"{np.mean(all_r2):.3f} $\pm$ {np.std(all_r2):.3f}"
])

headers = ["Pixel type", "A mean $\pm$ 1$\sigma$ [%]", "k mean $\pm$ 1$\sigma$ [s$^{-1}$]", f"$R^2$ mean $\pm$ 1$\sigma$"]
latex_table = tabulate(table_data, headers, tablefmt="latex")
with open(f"Data0{project_no}Obs00{observation}nrs{nrs_no}_fit_results_table.txt", "w") as f:
    f.write(latex_table)
with open(f"Data0{project_no}Obs00{observation}nrs{nrs_no}_fit_results_table_np.txt", "w") as f:
    f.write(tabulate(table_data, headers))
    
tabulate(table_data, headers)

In [ ]:
# Read the LaTeX table from file
with open(f"Data0{project_no}Obs00{observation}nrs{nrs_no}_fit_results_table.txt", "r") as f:
    content = f.read()

# Replace all occurrences of \textbackslash{}
cleaned_content = content.replace("textbackslash{}", "")
cleaned_content = cleaned_content.replace("\\$", "$")
cleaned_content = cleaned_content.replace("\^{}", "^")
cleaned_content = cleaned_content.replace("\{", "{")
cleaned_content = cleaned_content.replace("\}", "}")
cleaned_content = cleaned_content.replace("\\\_", "\\_")

# Write the cleaned content back to the file (or to a new file)
with open(f"Data0{project_no}Obs00{observation}nrs{nrs_no}_fit_results_table.txt", "w") as f:
    f.write(cleaned_content)


In [ ]:
import matplotlib.pyplot as plt

# Data
A_values = [37.284, 33.633, 38.144, 32.028]
A_err = [5.454, 6.762, 5.452, 6.275]
k_values = [0.831, 0.790, 0.907, 0.754]
k_err = [0.166, 0.173, 0.201, 0.091]
files = ['Data01224Obs002nrs1', 'Data01224Obs002nrs2', 'Data01224Obs003nrs1', 'Data01224Obs003nrs2']

# Plot A values
plt.figure(figsize=(10, 6))
plt.errorbar(files, A_values, yerr=A_err, fmt='o', color='blue', ecolor='black', capsize=2)
plt.ylabel('A Value (%)')
plt.title('A Values with Error Bars')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot k values
plt.figure(figsize=(10, 6))
plt.errorbar(files, k_values, yerr=k_err, fmt='o', color='green', ecolor='black', capsize=2)
plt.ylabel('k Value')
plt.title('k Values with Error Bars')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
tables = sorted(glob.glob(f'*_table_np.txt'))
print(tables)

In [ ]:
import glob
import pandas as pd

tables = sorted(glob.glob('*_table_np.txt'))

filename = tables[0]
df = pd.read_fwf(filename)

import re

def clean_latex_texttt(text):
    # Remove $ and \texttt{} wrappers
    # Example: $\texttt{HOT}$ -> HOT
    if isinstance(text, str):
        # Remove $...$
        text = re.sub(r'\$([^$]+)\$', r'\1', text)
        # Remove \texttt{...}
        text = re.sub(r'\\texttt\{([^}]+)\}', r'\1', text)
        return text.strip()
    return text

# Apply cleanup to the first column (Pixel type)
df.iloc[:, 0] = df.iloc[:, 0].apply(clean_latex_texttt)

df['Pixel type'] = df['Pixel type'].apply(clean_latex_texttt)
df = df[1:]

# Columns to split
cols_to_split = [
    'A mean $\pm$ 1$\sigma$ [%]',
    'k mean $\pm$ 1$\sigma$ [s$^{-1}$]',
    '$R^2$ mean $\pm$ 1$\sigma$'
]

for col in cols_to_split:
    # Split on the literal '$\pm$' (escaped)
    new_cols = df[col].str.split(r'\s*\$\\pm\$\s*', expand=True)
    df[col.replace(' mean $\pm$ 1$\sigma$', '_mean')] = new_cols[0].astype(float)
    df[col.replace(' mean $\pm$ 1$\sigma$', '_std')] = new_cols[1].astype(float)

# Drop the original combined columns if you want
df = df.drop(columns=cols_to_split)

df

In [ ]:
import glob
import pandas as pd
import re

def clean_latex_texttt(text):
    # Remove $ and \texttt{} wrappers, e.g. $\texttt{HOT}$ -> HOT
    if isinstance(text, str):
        text = re.sub(r'\$([^$]+)\$', r'\1', text)
        text = re.sub(r'\\texttt\{([^}]+)\}', r'\1', text)
        return text.strip()
    return text

def process_table(filename):
    df = pd.read_fwf(filename)
    
    # Clean first column Pixel type
    df.iloc[:, 0] = df.iloc[:, 0].apply(clean_latex_texttt)
    df = df[1:]
    
    # Fix column name if necessary (assuming first col is Pixel type)
    df.columns.values[0] = 'Pixel type'
    
    # Drop any header row repeats (if any)
    df = df[df['Pixel type'] != 'Pixel type']
    
    cols_to_split = [
        'A mean $\pm$ 1$\sigma$ [%]',
        'k mean $\pm$ 1$\sigma$ [s$^{-1}$]',
        '$R^2$ mean $\pm$ 1$\sigma$'
    ]
    
    for col in cols_to_split:
        if col in df.columns:
            new_cols = df[col].str.split(r'\s*\$\\pm\$\s*', expand=True)
            df[col.replace(' mean $\pm$ 1$\sigma$', '_mean')] = new_cols[0].astype(float)
            df[col.replace(' mean $\pm$ 1$\sigma$', '_std')] = new_cols[1].astype(float)
            df = df.drop(columns=[col])
    
    return df.reset_index(drop=True)

# Get list of all table files
tables = sorted(glob.glob('*_table_np.txt'))

# Process all tables and concatenate
all_dfs = [process_table(f) for f in tables]
combined_df = pd.concat(all_dfs, ignore_index=True)

print(combined_df)


In [ ]:
combined_df_new = combined_df.drop(combined_df.index[5::6])
combined_df_new

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Your DataFrame is called combined_df and looks like this:
# Columns: Pixel type, A_mean [%], A_std [%], ...

# Unique pixel types and assign colors
pixel_types = combined_df['Pixel type'].unique()[5:6]
colors = plt.cm.tab10.colors  # 10 distinct colors
color_map = {pt: colors[i % len(colors)] for i, pt in enumerate(pixel_types)}
group_size = 5
num_groups = len(combined_df) // group_size - 2

plt.figure(figsize=(12, 6))
point_spacing = 0.3     # smaller distance between points inside a group
group_spacing = 5       # larger distance between groups (files)

x_positions = np.arange(num_groups  + 1) * group_spacing

# But since pixel_index changes per pixel type, inside the loop:

for i, pixel_type in enumerate(pixel_types):
    pixel_index = i    
    x_pos = x_positions[:, None] + pixel_index * point_spacing  # pixel_index is scalar here
    df_pt = combined_df[combined_df['Pixel type'] == pixel_type].reset_index(drop=True)
    plt.errorbar(
        x_pos,
        df_pt['A_mean [%]'].to_numpy(),
        yerr=df_pt['A_std [%]'].to_numpy(),
        fmt='o',
        color=color_map[pixel_type],
        label=pixel_type,
        capsize=3,
        linestyle='None'
    )


# Compute all upper and lower bounds
k_mean = combined_df['A_mean [%]'][5::6].to_numpy()
k_std = combined_df['A_std [%]'][5::6].to_numpy()

upper_bounds = k_mean + k_std
lower_bounds = k_mean - k_std

# Find the min of upper bounds and max of lower bounds
lowest_upper = np.min(upper_bounds)
highest_lower = np.max(lower_bounds)

# Highlight the area
plt.axhspan(highest_lower, lowest_upper, color='lightblue', alpha=0.6)

    
plt.xticks(
    np.arange(num_groups + 1) * 5, 
    [f'{table[:19]}' for table in tables],
    rotation=45
)
plt.xlabel('Dataset')
plt.ylabel(f'$A$ mean [\%]')
#plt.title('A mean ± std per pixel type grouped by file')
plt.legend(title='Pixel type')
plt.tight_layout()
plt.savefig('Amean.pdf')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Your DataFrame is called combined_df and looks like this:
# Columns: Pixel type, A_mean [%], A_std [%], ...

# Unique pixel types and assign colors
pixel_types = combined_df['Pixel type'].unique()[5:6]
colors = plt.cm.tab10.colors  # 10 distinct colors
color_map = {pt: colors[i % len(colors)] for i, pt in enumerate(pixel_types)}
group_size = 5
num_groups = len(combined_df) // group_size - 2

plt.figure(figsize=(12, 6))
point_spacing = 0.3     # smaller distance between points inside a group
group_spacing = 5       # larger distance between groups (files)

x_positions = np.arange(num_groups  + 1) * group_spacing

# But since pixel_index changes per pixel type, inside the loop:

for i, pixel_type in enumerate(pixel_types):
    pixel_index = i    
    x_pos = x_positions[:, None] + pixel_index * point_spacing  # pixel_index is scalar here
    df_pt = combined_df[combined_df['Pixel type'] == pixel_type].reset_index(drop=True)
    plt.errorbar(
        x_pos,
        df_pt['k_mean [s$^{-1}$]'].to_numpy(),
        yerr=df_pt['k_std [s$^{-1}$]'].to_numpy(),
        fmt='o',
        color=color_map[pixel_type],
        label=pixel_type,
        capsize=3,
        linestyle='None'
    )


plt.xticks(
    np.arange(num_groups + 1) * 5, 
    [f'{table[:19]}' for table in tables],
    rotation=45
)

# Compute all upper and lower bounds
k_mean = combined_df['k_mean [s$^{-1}$]'][5::6].to_numpy()
k_std = combined_df['k_std [s$^{-1}$]'][5::6].to_numpy()

upper_bounds = k_mean + k_std
lower_bounds = k_mean - k_std

# Find the min of upper bounds and max of lower bounds
lowest_upper = np.min(upper_bounds)
highest_lower = np.max(lower_bounds)

# Highlight the area
plt.axhspan(highest_lower, lowest_upper, color='lightblue', alpha=0.6)


plt.xlabel('Dataset')
plt.ylabel(f'$k$ mean [s$^{{-1}}$]')
#plt.title('A mean ± std per pixel type grouped by file')
plt.legend(title='Pixel type')
plt.tight_layout()
plt.savefig('kmean.pdf')
plt.show()
